In [1]:
import h5py
import pandas as pd
from pyproj import Transformer
import os
import numpy as np
import pyvista as pv
import earthaccess
from sklearn.ensemble import IsolationForest
from matplotlib.colors import ListedColormap

In [2]:
#Not to run multiple times-> files can take moments to download
ul_lon = -92.03 #-92.03
lr_lon = -91.83 #-91.83
ul_lat = 32.59 #32.59
lr_lat = 32.37  #32.37

#downloading the files
l2a_results = earthaccess.search_data(
    short_name = "GEDI02_A", #l2A datasets
    temporal = ("2024-04-08","2024-11-02"),#time range-> 5 yrs
    bounding_box = (lr_lon, lr_lat, ul_lon, ul_lat), #area of study
    count = 2
)
#downloading only the ones that are not in the direc
path = "C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files"
for file in l2a_results:
    if file not in os.listdir(path):
        l2a_files = earthaccess.download(file, path)
    else:
        print("File already in the directory....")

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
folder_path = "C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files\\"
files = [(folder_path+ file) for file in os.listdir(folder_path)][1:]
files[16:]

['C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files\\GEDI02_A_2022097224744_O18797_03_T09874_02_003_02_V002.h5',
 'C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files\\GEDI02_A_2022098002035_O18798_03_T07029_02_003_02_V002.h5',
 'C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files\\GEDI02_A_2022098015326_O18799_02_T05607_02_003_02_V002.h5',
 'C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files\\GEDI02_A_2022098032616_O18800_02_T02762_02_003_02_V002.h5',
 'C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files\\GEDI02_A_2022118153019_O19118_03_T10303_02_003_02_V002.h5',
 'C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files\\GEDI02_A_2022157001845_O19713_03_T07457_02_003_02_V002.h5',
 'C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files\\GEDI02_A_2022218001000_O20659_03_T00342_02_003_02_V002.h5',
 'C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files\\GEDI02_A_2022224135830_O20761_02_T09273_02_003_02_V002.h5',
 'C:\\Users\\Admin\\Documents\\Srijan Baniya\\L2A files\\GEDI02_

In [4]:
def convert_to_dataframe(file)-> pd.DataFrame:
    beams = [g for g in file.keys() if g.startswith("BEAM")]    
    collected_data = []
    for beam in beams:
        df = pd.DataFrame({
            "Shot_num": file[beam]['shot_number'][:],
            "latitude": file[beam]['lat_lowestmode'][:],
            "longitude": file[beam]['lon_lowestmode'][:],
            "elev_highest": file[beam]['elev_highestreturn'][:],
            "elev_lowest": file[beam]['elev_lowestmode'][:],
            "rh25": file[beam]['rh'][:, 25],
            "rh50": file[beam]['rh'][:, 50],
            "rh60": file[beam]['rh'][:, 60],
            "rh75": file[beam]['rh'][:, 75],
            "rh90": file[beam]['rh'][:,90],
            "rh98": file[beam]['rh'][:, 98],
            "quality_flag": file[beam]['quality_flag'][:],
            "degrade_flag": file[beam]['degrade_flag'][:]
        })

        collected_data.append(df)
    df = pd.concat(collected_data, ignore_index = True) if collected_data else pd.DataFrame()

    #filtering out the invalid shots
    df_filtered = df[(df['quality_flag']==1)&(df['degrade_flag']!=1)].copy()

    #clipping to our study area only
    min_lat, max_lat = 32.37, 32.59  #32.37, 32.59
    max_lon, min_lon = -91.83, -92.03 #-91.83, -92.03
    df_clipped = df_filtered[(df_filtered['latitude']>=min_lat)& (df_filtered['latitude']<= max_lat) &(df_filtered['longitude']>=min_lon) & (df_filtered['longitude']<=max_lon)].copy()

    #Transssforming latitude, longitude into x,y
    #global to local
    ESPG_WGS84 = 4326
    ESPG_UTM15 = 32615
    tx = Transformer.from_crs(ESPG_WGS84, ESPG_UTM15, always_xy = True)

    X, Y = tx.transform(df_clipped['longitude'], df_clipped['latitude'])
    df_clip = df_clipped.copy()
    df_clip['X'], df_clip['Y'] = X,Y
    df_clip.drop(columns = ['latitude','longitude','quality_flag','degrade_flag'], inplace = True)
    ordered_cols = ['Shot_num','X','Y','elev_highest','elev_lowest','rh25','rh50','rh60','rh75','rh90','rh98']
    df_transformed = df_clip[ordered_cols].copy()
    return df_transformed

In [5]:
collected_data = []
for f in files:
    try:
        file = h5py.File(f)
        df = convert_to_dataframe(file)
        collected_data.append(df)
    except Exception:
        continue

df = pd.concat(collected_data, ignore_index = True) if collected_data else pd.DataFrame()
df.head()

,Shot_num,X,Y,elev_highest,elev_lowest,rh25,rh50,rh60,rh75,rh90,rh98
0,144770000200092078,596566.041361,3.596716e+06,-3.205212,-6.389374,-1.16,-0.14,0.18,0.78,1.57,2.47
1,144770000200092079,596605.958813,3.596757e+06,-2.834203,-6.055826,-1.12,-0.14,0.18,0.74,1.53,2.47
2,144770000200092085,596845.473546,3.597001e+06,-3.125966,-6.572353,-1.08,-0.11,0.26,0.86,1.68,2.65
3,144770000200092100,597443.916933,3.597612e+06,8.690321,-6.593659,6.44,9.14,9.92,11.12,12.69,14.27
4,144770000200092101,597483.511759,3.597652e+06,9.449334,5.291193,-2.24,-0.59,0.00,0.93,2.09,3.33


In [6]:
#Running isolation forest to filter out the outliers
cols = df.drop(columns = ['Shot_num']).columns
iso_forest = IsolationForest(contamination = 0.03, random_state = 42)
df['outliers'] = iso_forest.fit_predict(df[cols].copy())#outliers are -1 and non-outliers are stored as 1
df['anomaly_score'] = iso_forest.decision_function(df[cols].copy())
outliers = df[df['outliers']!=1] 
df = df[df['outliers'] ==1]
outliers

,Shot_num,X,Y,elev_highest,elev_lowest,rh25,rh50,rh60,rh75,rh90,rh98,outliers,anomaly_score
14,144770000200092229,602593.101802,3.602861e+06,21.335400,12.045134,3.44,5.390000,5.990000,6.810000,7.790000,8.760000,-1,-0.029053
16,144770000200092231,602673.621315,3.602943e+06,24.337938,-7.803389,14.42,24.270000,25.730000,27.530001,29.360001,31.010000,-1,-0.072472
34,144770100200088385,601702.525163,3.601046e+06,16.968071,10.674910,0.82,2.920000,3.480000,4.300000,5.430000,6.060000,-1,-0.004532
42,144770100200088431,603540.811861,3.602919e+06,24.030617,-7.997078,15.62,23.860001,25.770000,27.750000,29.700001,31.270000,-1,-0.075440
49,144770100200088441,603940.330994,3.603326e+06,18.111721,9.533544,1.91,3.850000,4.450000,5.390000,6.850000,8.160000,-1,-0.029578
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20532,319501100300197790,600869.409391,3.593022e+06,22.694485,-7.987467,13.36,20.600000,22.240000,24.330000,26.570000,28.889999,-1,-0.012534
20534,319501100300197792,600951.089959,3.592942e+06,23.324934,-8.290168,15.75,21.760000,23.469999,25.860001,28.139999,30.150000,-1,-0.055438
20535,319501100300197793,600991.926578,3.592902e+06,23.167103,-8.671955,13.06,21.350000,23.209999,25.379999,27.799999,30.230000,-1,-0.034781
20536,319501100300197794,601032.765710,3.592862e+06,22.783197,-9.130512,9.25,19.820000,22.799999,25.750000,28.330000,30.379999,-1,-0.022725


In [14]:
#df = df.drop(columns = ['outliers', 'anomaly_score'])
xy = df[['X','Y']].to_numpy()

In [15]:
def to_polyData(xyz: np.ndarray)-> pv.PolyData:
    return pv.PolyData(xyz)

In [16]:
#A 25m footprint may have these points anywhere inside the radius not only in the center
def random_offsets(n, radius = 12.5):
    r = radius * np.sqrt(np.random.rand(n))
    theta = 2 * np.pi * np.random.rand(n)
    dx = r * np.cos(theta)
    dy = r * np.sin(theta)
    return dx, dy

In [17]:
#canopy top
dx, dy = random_offsets(len(xy))
xy_jittered = xy + np.c_[dx,dy] #this brings randomness but that is within the circle only
z_exg = 50.0
z = df['elev_highest'].to_numpy() * z_exg
top_xyz = np.c_[xy_jittered, z].astype(np.float32)
poly_top = to_polyData(top_xyz)

In [18]:
#Mid canopy
xy_rep = np.repeat(xy, 6, axis = 0)
dx, dy = random_offsets(len(xy_rep))
xy_jittered = xy_rep + np.c_[dx, dy]
z_exg = 50.0
z = np.r_[
    (df['elev_lowest'] + df['rh25']).to_numpy(),
    (df['elev_lowest'] + df['rh50']).to_numpy(),
    (df['elev_lowest'] + df['rh60']).to_numpy(),
    (df['elev_lowest'] + df['rh75']).to_numpy(),
    (df['elev_lowest'] + df['rh90']).to_numpy(),
    (df['elev_lowest'] + df['rh98']).to_numpy()
] * z_exg

mid_xyz = np.c_[xy_jittered, z].astype(np.float32)
poly_mid = to_polyData(mid_xyz)
mid_xyz.shape

(120798, 3)

In [19]:
#canopy ground
dx, dy = random_offsets(len(xy))
xy_jittered = xy + np.c_[dx,dy]
z_exg = 50.0
z = df['elev_lowest'].to_numpy() * z_exg
ground_xyz = np.c_[xy_jittered, z].astype(np.float32)

poly_ground = to_polyData(ground_xyz)

In [20]:
#scalar labels 
levels = np.r_[
    np.full(len(df),15),
    np.full(len(df),25),
    np.full(len(df),50),
    np.full(len(df),60),
    np.full(len(df),75),
    np.full(len(df),98)
]

#Hex codes
greens = [
    "#BDE6CF", #rh25
    "#4FA76C", #rh50
    "#4FA76C", #rh60
    "#4FA76C", #rh75
    "#228B22", #rh90
    "#008000" #rh98
]

cmap_vert = ListedColormap(greens)

In [21]:
xmin, xmax = df['X'].min(), df['X'].max()
ymin, ymax = df['Y'].min(), df['Y'].max()

aoi_xy = np.array([[xmin, ymin], [xmax, ymin], [xmax, ymax],[xmin, ymax], [xmin, ymin]])
aoi_z = np.full((aoi_xy.shape[0] , 1), df['elev_lowest'].min() * z_exg)
aoi_poly = to_polyData(np.c_[aoi_xy, aoi_z])

In [22]:
#Setting the backend to trame
pv.set_jupyter_backend('trame')

#pyvista object plotter
plotter = pv.Plotter(window_size = [1200,900], notebook = True)
plotter.add_axes()

#Ground points
plotter.add_points(
    poly_ground,
    color = "#8B4513", #brown
    point_size = 5.0,
    render_points_as_spheres = True,
    label = "Ground",
    pickable = True
)

#Canopy top
plotter.add_points(
    poly_top,
    color = "#008000",
    point_size = 5.0,
    render_points_as_spheres = True,
    label = "Canopy Top",
    pickable = True
)

#Mid canopy
plotter.add_points(
    poly_mid,
    point_size = 5.0,
    render_points_as_spheres = True,
    scalars = levels,
    cmap = cmap_vert,
    pickable = True,
    clim = (49,99),
    scalar_bar_args = dict(
        title = "Relative Height (%)",
        n_labels = 3
    ),
    name = "Vertical jittered RH points"
)

plotter.add_mesh(aoi_poly, color = "#444444", line_width = 2, render_lines_as_tubes = True)
plotter.show_grid(
    location = 'outer',
    xtitle = 'Easting (m, UTM 15N)',
    ytitle = 'Northing (m, UTM 15N)',
    ztitle = 'Exaggerated z'
)
plotter.add_axes()
plotter.enable_point_picking(callback =None, show_message = True, use_picker = True)
plotter.add_legend(
    labels = [
        ("Ground", "#8B4513"),
        ("rh25", "#BDE6CF"),
        ("rh50", "#4FA76C"),
        ("rh60", "#4FA76C"),
        ("rh75", "#4FA76C"),
        ("rh90", "#228B22"),
        ("rh98", "#008000"),
        ("Canopy Top", "#008000")
    ],
    border = False, bcolor = None
)

plotter.show()
    

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x25fe48d1940_0" src="http://localhost:8888/trame-…